In [3]:
import os
import scipy.io as sio 

In [30]:
p = r"E:\datasources\MS10Data\1\psg\psg.mat"
rdp=r"E:\datasources\MS10Data\1\raw\rawMat.mat"

In [11]:
psg1 = sio.loadmat(p)

In [29]:
psg1["psgStage"].shape

(1, 1116)

In [19]:
psg1

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Dec 12 16:29:57 2018',
 '__version__': '1.0',
 '__globals__': [],
 'psgStage': array([[(array([[737201.88489583]]), array([[3.1]])),
         (array([[737201.88524305]]), array([[3.1]])),
         (array([[737201.88559027]]), array([[3.1]])), ...,
         (array([[737202.27135207]]), array([[1.1]])),
         (array([[737202.27169929]]), array([[1.1]])),
         (array([[737202.27204651]]), array([[1.1]]))]],
       dtype=[('Time', 'O'), ('stage', 'O')])}

In [31]:
rs= sio.loadmat(rdp)

In [64]:
rs['rawData'][0,0][0]

array([[8608],
       [8408],
       [8216],
       ...,
       [7760],
       [7752],
       [7748]], dtype=uint16)

In [55]:
rs['rawData'].dtype

dtype([('dataSignalOld', 'O'), ('dataSignal', 'O'), ('dataSwitch', 'O'), ('Time', 'O')])

In [50]:
tmp=rs['rawData'][0][0]

In [51]:
len(tmp)

4

In [54]:
tmp

(array([[8608],
       [8408],
       [8216],
       ...,
       [7760],
       [7752],
       [7748]], dtype=uint16), array([[8608],
       [8408],
       [8216],
       ...,
       [7760],
       [7752],
       [7748]], dtype=uint16), array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=uint8), array([[737201.88136574],
       [737201.88136579],
       [737201.88136583],
       ...,
       [737202.29539114],
       [737202.29539118],
       [737202.29539123]]))